In [1]:
%load_ext autoreload
%autoreload 2
%cd "/home/hew/python/contp"
%ls

/home/hew/python/contp
ckpt/  dataset/      dataset.zip  model/     script/  utils/
data/  dataset_bak/  generation/  README.md  temp/


/home/hew/anaconda3/envs/contp/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
import re
import numpy as np
import pandas as pd
from utils.file import list_file, get_basename, read_fasta
from utils.bio import diversity, novelty

In [3]:
dir = './generation/'
files = list_file(dir, absolute=True)
files

['./generation/esm3_transporter_IPR044677.fasta',
 './generation/proteindt_IPR044677_sample1000.fasta',
 './generation/esm3_IPR047818_mask80_pred.csv',
 './generation/proteindt_IPR004670_sample1000.fasta',
 './generation/diversity_df.csv',
 './generation/progen2_IPR044677_10p_clean_pred.csv',
 './generation/esm3_IPR003445_mask80.fasta',
 './generation/progen2_IPR047818_10p_clean_pred.csv',
 './generation/esm3_transporter_IPR047818.fasta',
 './generation/esm3_IPR044677_mask80.fasta',
 './generation/esm3_transporter_IPR003445.fasta',
 './generation/progen2_IPR044677_10p_clean.fasta',
 './generation/esm3_transporter_IPR004670_pred.csv',
 './generation/progen2_IPR004670_10p_clean.fasta',
 './generation/esm3_IPR004670_mask80_pred.csv',
 './generation/proteindt_IPR004670_sample1000_pred.csv',
 './generation/proteindt_IPR003445_sample1000_pred.csv',
 './generation/progen2_IPR004670_10p_clean_pred.csv',
 './generation/esm3_IPR004670_mask80.fasta',
 './generation/esm3_IPR003445_mask80_pred.csv'

In [4]:
finished_basenames = []
for file in files:
    if file.endswith('csv'):
        basename = get_basename(file)
        finished_basenames.append(basename.replace('_pred', ''))

todo_basenames = []
for file in files:
    if not file.endswith('fasta'):
        continue

    basename = get_basename(file)
    if basename not in finished_basenames:
        todo_basenames.append(basename)

len(todo_basenames), todo_basenames

(0, [])

In [5]:
model = 'progen2'
esmfold_df = pd.read_csv(dir + f'{model}_esmfold_summary.csv')
esmfold_df

,fasta,seq_id,length,pLDDT,pTM,pdb_path
0,progen2_IPR003445_10p_clean,N1U6P9,127,81.70,0.628,/ibex/project/c2108/junbo/results/generation-r...
1,progen2_IPR003445_10p_clean,A0A2M8H6J4,128,49.44,0.329,/ibex/project/c2108/junbo/results/generation-r...
2,progen2_IPR003445_10p_clean,A0A399R6R2,128,53.54,0.386,/ibex/project/c2108/junbo/results/generation-r...
3,progen2_IPR003445_10p_clean,A0ABV2TZV0,131,53.21,0.446,/ibex/project/c2108/junbo/results/generation-r...
4,progen2_IPR003445_10p_clean,A0A0C2KA81,132,49.33,0.322,/ibex/project/c2108/junbo/results/generation-r...
...,...,...,...,...,...,...
3775,progen2_IPR047818_10p_clean,A0ABQ1GVA4,511,56.74,0.478,/ibex/project/c2108/junbo/results/generation-r...
3776,progen2_IPR047818_10p_clean,A0A379F8Y2,511,71.52,0.792,/ibex/project/c2108/junbo/results/generation-r...
3777,progen2_IPR047818_10p_clean,A0A730JYW4,511,50.82,0.510,/ibex/project/c2108/junbo/results/generation-r...
3778,progen2_IPR047818_10p_clean,A0A5I2DIB3,511,57.66,0.494,/ibex/project/c2108/junbo/results/generation-r...


In [6]:
esmfold_df.groupby('fasta').agg({
    'pLDDT': ['mean'],
    'pTM': ['mean'],
    'length': ['mean']
})

,pLDDT,pTM,length
,mean,mean,mean
fasta,,,
progen2_IPR003445_10p_clean,68.870470,0.588006,457.558000
progen2_IPR004670_10p_clean,80.918840,0.774201,437.174000
progen2_IPR044677_10p_clean,51.256650,0.336355,469.310000
progen2_IPR047818_10p_clean,76.839141,0.806815,455.915385


In [7]:
esmfold_info = []
for model in ['progen2', 'esm3', 'proteindt']:
    esmfold_df = pd.read_csv(dir + f'{model}_esmfold_summary.csv')
    esmfold_df = esmfold_df.groupby('fasta').agg({
        'pLDDT': ['mean'],
        'pTM': ['mean'],
        'length': ['mean']
    })
    esmfold_df['model'] = 'mask80' if model == 'esm3' else model
    esmfold_info.append(esmfold_df)

esmfold_info = pd.concat(esmfold_info)
esmfold_info['ipr'] = esmfold_info.index.map(lambda x: re.search(r'IPR\d+', x).group(0))
esmfold_info

,pLDDT,pTM,length,model,ipr
,mean,mean,mean,,
fasta,,,,,
progen2_IPR003445_10p_clean,68.870470,0.588006,457.558000,progen2,IPR003445
progen2_IPR004670_10p_clean,80.918840,0.774201,437.174000,progen2,IPR004670
progen2_IPR044677_10p_clean,51.256650,0.336355,469.310000,progen2,IPR044677
progen2_IPR047818_10p_clean,76.839141,0.806815,455.915385,progen2,IPR047818
esm3_IPR003445_mask80,80.969152,0.822516,494.618999,mask80,IPR003445
esm3_IPR004670_mask80,85.284228,0.853005,419.259146,mask80,IPR004670
esm3_IPR044677_mask80,68.408929,0.636364,329.118182,mask80,IPR044677
esm3_IPR047818_mask80,68.531819,0.730240,499.322613,mask80,IPR047818


In [8]:
IPR_list = []
for file in files:
    if file.endswith('csv'):
        basename = get_basename(file)
        match = re.search(r'IPR\d+', basename)
        if match:
            ipr = match.group(0)
            IPR_list.append(ipr)
IPR_list = sorted(set(IPR_list))
IPR_list

['IPR003445', 'IPR004670', 'IPR044677', 'IPR047818']

In [9]:
IPR2substrate = {
    'IPR003445': ['cation:potassium', 'cation:sodium'],
    'IPR004670': ['cation:sodium', 'cation:proton'],
    'IPR044677': ['anion:phosphate'],
    'IPR047818': ['anion:phosphate'],
}

In [10]:
summary = []
for model in ['progen2', 'proteindt', 'esm3_transporter', 'mask80']:
    for ipr in IPR2substrate:
        for file in files:
            if file.endswith('csv') and model in file and ipr in file:
                pred_df = pd.read_csv(file)
                match_df = pred_df['substrate_top1'].isin(IPR2substrate[ipr])
                acc = match_df.sum() / len(match_df)
                sel_df = esmfold_info[(esmfold_info['model'] == model) & (esmfold_info['ipr'] == ipr)]
                if len(sel_df) > 0:
                    plddt = sel_df['pLDDT'].values[0][0]
                    ptm = sel_df['pTM'].values[0][0]
                    length = sel_df['length'].values[0][0]
                else:
                    plddt = np.nan
                    ptm = np.nan
                    length = np.nan

                row = [ipr, model, acc, plddt, ptm, length]
                summary.append(row)

summary = pd.DataFrame(summary, columns=['ipr', 'model', 'top1_acc', 'plddt', 'ptm', 'length'])
summary = summary.sort_values(by=['ipr', 'model'], ascending=[True, True])
summary = summary.reset_index(drop=True)
summary.to_csv('./generation/summary.csv', index=False)
summary

,ipr,model,top1_acc,plddt,ptm,length
0,IPR003445,esm3_transporter,0.006,NaN,NaN,NaN
1,IPR003445,mask80,0.770,80.969152,0.822516,494.618999
2,IPR003445,progen2,0.324,68.870470,0.588006,457.558000
3,IPR003445,proteindt,0.909,87.965165,0.902445,495.345035
4,IPR004670,esm3_transporter,0.108,NaN,NaN,NaN
5,IPR004670,mask80,0.904,85.284228,0.853005,419.259146
6,IPR004670,progen2,0.756,80.918840,0.774201,437.174000
7,IPR004670,proteindt,0.983,90.842700,0.906919,419.823000
8,IPR044677,esm3_transporter,0.003,NaN,NaN,NaN
9,IPR044677,mask80,0.137,68.408929,0.636364,329.118182


In [11]:
def calculate_diversity(fasta):
    seqs, _ = read_fasta(fasta)
    return diversity(seqs)


diversity_df = []
for model in ['progen2', 'proteindt', 'esm3_transporter', 'mask80']:
    for ipr in IPR2substrate:
        for file in files:
            if file.endswith('fasta') and model in file and ipr in file:
                value = calculate_diversity(file)
                diversity_df.append([model, ipr, value])
diversity_df = pd.DataFrame(diversity_df, columns=['model', 'ipr', 'value'])
diversity_df.to_csv('./generation/diversity_df.csv', index=False)
diversity_df

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

,model,ipr,value
0,progen2,IPR003445,0.886
1,progen2,IPR004670,0.729
2,progen2,IPR044677,0.900
3,progen2,IPR047818,0.597
4,proteindt,IPR003445,0.844
5,proteindt,IPR004670,0.713
6,proteindt,IPR044677,0.716
7,proteindt,IPR047818,0.218
8,esm3_transporter,IPR003445,0.889
9,esm3_transporter,IPR004670,0.893


In [12]:
diversity_df = diversity_df.sort_values(by=['ipr', 'model'], ascending=[True, True])
diversity_df

,model,ipr,value
8,esm3_transporter,IPR003445,0.889
12,mask80,IPR003445,0.819
0,progen2,IPR003445,0.886
4,proteindt,IPR003445,0.844
9,esm3_transporter,IPR004670,0.893
13,mask80,IPR004670,0.679
1,progen2,IPR004670,0.729
5,proteindt,IPR004670,0.713
10,esm3_transporter,IPR044677,0.907
14,mask80,IPR044677,0.891


In [ ]:
# def calculate_novelty(fasta):
#     seqs, _ = read_fasta(fasta)
#     return novelty(seqs)
#
#
# novelty_df = []
# for model in ['progen2', 'proteindt', 'esm3_transporter', 'mask80']:
#     for ipr in IPR2substrate:
#         for file in files:
#             if file.endswith('fasta') and model in file and ipr in file:
#                 value = calculate_diversity(file)
#                 novelty_df.append([model, ipr, value])
# novelty_df = pd.DataFrame(novelty_df, columns=['model', 'ipr', 'value'])
# novelty_df = novelty_df.sort_values(by=['ipr', 'model'], ascending=[True, True])
# novelty_df.to_csv('./generation/novelty_df.csv', index=False)
# novelty_df